# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [55]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
#import cartopy
#import geoviews
#import pyproj

# Import API key
from api_keys import geoapify_key

In [56]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,25.99,69,84,4.12,CC,1722877047
1,1,sungai penuh,-2.0833,101.3833,17.85,95,89,2.39,ID,1722877048
2,2,ilulissat,69.2167,-51.1000,7.05,63,5,1.29,GL,1722877003
3,3,port-aux-francais,-49.3500,70.2167,0.72,68,20,4.11,TF,1722876996
4,4,mumbwa,-14.9783,27.0619,22.40,13,1,1.30,ZM,1722877052


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [57]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5
)

# Display the map

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [58]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_spots =  city_data_df[(city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 27) & (city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
vacation_spots = vacation_spots.dropna(axis=0)

# Display sample data
vacation_spots

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,tolanaro,-25.0319,46.9987,22.04,83,0,2.06,MG,1722877069
59,59,kigoma,-4.8769,29.6267,26.33,57,0,1.03,TZ,1722877118
173,173,vaasa,63.0960,21.6158,21.14,65,0,4.12,FI,1722877250
180,180,kitimat,54.0524,-128.6534,25.51,48,0,0.89,CA,1722877258
319,319,namsos,64.4662,11.4957,24.95,47,0,4.12,NO,1722877392
446,446,letlhakane,-21.4167,25.5833,21.31,15,0,3.77,BW,1722877571
465,465,haiku-pauwela,20.9219,-156.3051,23.47,81,0,3.13,US,1722877498
531,531,prado,-17.3411,-39.2208,25.17,64,0,3.72,BR,1722877588
539,539,immenhausen,51.4276,9.4802,23.58,53,0,3.60,DE,1722877675
566,566,alta,69.9689,23.2717,24.59,36,0,3.60,NO,1722877706


### Step 3: Create a new DataFrame called `hotel_df`.

In [59]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_spots.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
17,17,tolanaro,-25.0319,46.9987,22.04,83,0,2.06,MG,1722877069,
59,59,kigoma,-4.8769,29.6267,26.33,57,0,1.03,TZ,1722877118,
173,173,vaasa,63.0960,21.6158,21.14,65,0,4.12,FI,1722877250,
180,180,kitimat,54.0524,-128.6534,25.51,48,0,0.89,CA,1722877258,
319,319,namsos,64.4662,11.4957,24.95,47,0,4.12,NO,1722877392,
446,446,letlhakane,-21.4167,25.5833,21.31,15,0,3.77,BW,1722877571,
465,465,haiku-pauwela,20.9219,-156.3051,23.47,81,0,3.13,US,1722877498,
531,531,prado,-17.3411,-39.2208,25.17,64,0,3.72,BR,1722877588,
539,539,immenhausen,51.4276,9.4802,23.58,53,0,3.60,DE,1722877675,
566,566,alta,69.9689,23.2717,24.59,36,0,3.60,NO,1722877706,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [60]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    #latitude = hotel_df.loc[index, "Lat"]
    #longitude = hotel_df.loc[index, "Lng"]
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tolanaro - nearest hotel: Hôtel Mahavokey
kigoma - nearest hotel: sunset vista hotel
vaasa - nearest hotel: Sokos Hotel Vaakuna
kitimat - nearest hotel: MStar Hotel
namsos - nearest hotel: Tinos hotell
letlhakane - nearest hotel: Boteti Hotel
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
prado - nearest hotel: Pousada Recanto do Prado
immenhausen - nearest hotel: No hotel found
alta - nearest hotel: Scandic Alta
chicoana - nearest hotel: Hostal Los Faroles


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
17,17,tolanaro,-25.0319,46.9987,22.04,83,0,2.06,MG,1722877069,Hôtel Mahavokey
59,59,kigoma,-4.8769,29.6267,26.33,57,0,1.03,TZ,1722877118,sunset vista hotel
173,173,vaasa,63.0960,21.6158,21.14,65,0,4.12,FI,1722877250,Sokos Hotel Vaakuna
180,180,kitimat,54.0524,-128.6534,25.51,48,0,0.89,CA,1722877258,MStar Hotel
319,319,namsos,64.4662,11.4957,24.95,47,0,4.12,NO,1722877392,Tinos hotell
446,446,letlhakane,-21.4167,25.5833,21.31,15,0,3.77,BW,1722877571,Boteti Hotel
465,465,haiku-pauwela,20.9219,-156.3051,23.47,81,0,3.13,US,1722877498,Inn At Mama's Fish House
531,531,prado,-17.3411,-39.2208,25.17,64,0,3.72,BR,1722877588,Pousada Recanto do Prado
539,539,immenhausen,51.4276,9.4802,23.58,53,0,3.60,DE,1722877675,No hotel found
566,566,alta,69.9689,23.2717,24.59,36,0,3.60,NO,1722877706,Scandic Alta


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [61]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)